# Integrating datasets

## Introduction

* Why integrate?
    * Batch correction vs data integration
* What is integration?
    * Principles and challenges paper
    * Batch vs bio
* Benchmarks
    * scIB
    * Ocular atlas
    * Hemberg
    * Tran
    * Others
* Different output types
    * Features
    * Embedding
    * Graph
* General pros/cons
* With existing labels
* Projecting onto a reference

In [ ]:
import scanpy as sc
import scvi
import bbknn
import scib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

## Dataset

The dataset we will use to demonstrate data integration contains several samples from the pancreas sequenced using different scRNA-seq technologies and protocols. First let's download the data from Figshare. This will create a file caled `scIB-pancreas.h5ad` in the `datasets/` directory. Please modify the command if you would prefer to download somewhere else. You can also download the file using your browser with this [link](https://figshare.com/ndownloader/files/24539828).

In [ ]:
!wget -nc https://figshare.com/ndownloader/files/24539828 -O ../../datasets/scIB-pancreas.h5ad

Once we have downloaded the file we can read it using **scanpy** to get an `AnnData` object.

In [ ]:
adata = sc.read_h5ad("../../datasets/scIB-pancreas.h5ad")
adata.layers["logcounts"] = adata.X
adata

The original dataset contains 16382 cells and measurements for 19093 cells. There are two versions of the expression matrix, `counts` which contains the raw count values and `logcounts` which contains **{scran}**-normalised log-counts (these values are also stored in `adata.X`).

The `obs` slot contains three variables:

* `tech` - The sequencing technology used for each cell. For some technologies there are more than one sample. We will use this as the batch variable for our integration.
* `celltype` - The cell identity label for each cell
* `size_factors` - The normalisation size factors calculated by **{scran}**

Let's have a look at the different batches and how many cells we have for each.

In [ ]:
adata.obs["tech"].value_counts()

There are nine different samples from five different technologies. For simplicity we will select four samples to use for this section, two collected using the inDrop platform and two from the celseq protocol.

In [ ]:
keep_batches = ["inDrop1", "inDrop2", "celseq", "celseq2"]
adata = adata[adata.obs["tech"].isin(keep_batches)].copy()
adata

Most integration methods require a single object containing all the samples and a batch variable (like we have here). If instead you have separate objects for each of your samples you can join them using the **anndata** `concat()` function. See the the [concatenation tutorial](https://anndata.readthedocs.io/en/stable/concatenation.html) for more details.

> **Integrating UMI and full-length data**
>
> Integrating samples from UMI and full-length protocols can present additional challenges. This is because full-length protocols are affected by gene-length bias (longer genes will be more highly expressed) while UMI data is not [10.12688/f1000research.11290.1].
> Because of this it is generally recommended to transform counts for full-length samples into a unit which corrects for gene-length (such as transcripts per million (TPM)) before attempting integration.
>
> This case applies to the samples we use here.
> The celseq protocol produces full-length data but the values we use here are inferred UMI counts calculated by the authors of the original study.
> This conversion process produces floating point numbers rather than integers which results in warnings in some of the following sections.

## Unintegrated

It is always recommended to look at the raw data before performing any integration. This can give some indication of how big any batch effects are and what might be causing them. For some experiments it might even suggest that integration is not required if samples already overlap. This is not uncommon for mouse or cell line studies from a single lab for example where most of the variables which contribute to batch effects can be controlled.

We will perform variable gene selection, PCA and UMAP dimensionality reduction as we have seen previously in the pre-processing chapter.

In [ ]:
sc.pp.highly_variable_genes(adata)
sc.tl.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)
adata.obsm["X_raw_umap"] = adata.obsm["X_umap"]
adata

This adds several new items to our AnnData object. The `var` slot now includes means, dispersions and the selected variable genes. In the `obsp` slot we have distances and connectivities for our KNN graph and in `obsm` are the PCA and UMAP embeddings. We also store the UMAP as `X_raw_umap` as we want to compare it to the integrated embeddings later on.

Let's plot the UMAP, colouring the points by cell identity and batch labels. If the dataset had not already been labelled (which is often the case) we would only be able to consider the batch labels.

In [ ]:
label_key = "celltype"
batch_key = "tech"
sc.pl.umap(adata, color=[label_key, batch_key])

These plots clearly show that there are batch effects in this dataset. The two different technologies (cellseq and inDrop) are clearly separated and even for one technology there are differences between samples. If we consider the cell identity labels we see that cells which have the same label are not always located near each other. If we were to perform a clustering analysis using this raw data we would probably end up with different clusters for each sample/identity combination which would be difficult to interpret at the annotation stage. We are also likely to overlook rare cell types which are not common enough in any single sample to produce their own cluster.

Now that we have confirmed there are batch effects to correct we can move on to the different integration methods.

## Batch-aware feature selection

For similar reasons to when only working with one sample we want to select a subset of genes to use for our analysis. However, when integrating multiple samples it is important that gene selection is performed in a batch-aware way. This is because there may be a small cell population which is only present in one batch. Important genes for this population might not be variable across the whole datasets but should be when only considering one batch.

We can perform batch aware highly variable gene selection by setting the `batch_key` argument. **scanpy** will then calculate HVGs for each batch separately and combine the results.

In [ ]:
sc.pp.highly_variable_genes(adata, n_top_genes=2000, flavor="cell_ranger", batch_key=batch_key)
adata
adata.var

We can see there are now some addtional columns in `var`:

* `highly_variable_nbatches` - The number of batches where each gene was found to be highly variable
* `highly_variable_intersection` - Whether each genes was highly variable in every batch
* `highly_variable` - Whether each genes was selected as highly variable after combining the results from each batch 

Let's check how many batches each gene was variable in.

In [ ]:
ax = adata.var.hist(column="highly_variable_nbatches")

In this case most genes are variable in all batches but it is still good practice to perform batch-aware gene selection in case they aren't.

> **How many genes to use?**
>
> This is a question which doesn't have a clear answer. The authors of the **scvi-tools** package which we use below recommend between 1000 and 10000 genes but how many depends on the context including the complexity of the dataset and the number of batches.
> In general it is better to select slightly too many genes than to select too few and risk removing genes which are important for a rare cell type.
> However, more genes will also increase the time required to run the integration methods.

We will create an object with just the selected genes to use for integration.

In [ ]:
adata_hvg = adata[:, adata.var["highly_variable"]].copy()
adata_hvg

## Integrating with scVI

The first integration method we will use is **scVI**. This method is based on a conditional variational autoencoder, a type of neural network which attempts to reconstruct the distribution of a dataset while considering differences between conditions (in this case batches). In benchmarking studies **scVI** has been shown to perform well across a range of datasets with a good balance of batch correction while conserving biological variability [REF]. **scVI** models raw counts directly so it is important that we provide it with a counts matrix rather than a normalised expression matrix.

First let's make a copy of our dataset to use for this integration. It's not normally necessary to do this but as we will demonstrate multiple integration methods making a copy makes it easier to show what has been added by each method.

In [ ]:
adata_scvi = adata_hvg.copy()

### Data preparation

The first step in using **scVI** is prepare our AnnData object. This step stores some information required by **scVI** such as which expression matrix to use and what the batch key is.

In [ ]:
scvi.model.SCVI.setup_anndata(adata_scvi, layer="counts", batch_key=batch_key)
adata_scvi

The fields created byh **scVI** are prefixed with `_scvi`. These are designed for internal use and should not be manually modified. Also note the general advice from **scVI** not to make any changes to our object until after the model is trained.

**scVI** also provides a handy function for viewing the current setup.

In [ ]:
scvi.data.view_anndata_setup(adata_scvi)

Here we can see the number of batches, the labels for those batches and how **scVI** has encoded them. It is important to check this information to double check everything is as intended.

### Building the model

We can now construct an **scVI** model object. As well as the **scVI** model we use here the **scvi-tools** package contains various other models (we will use the **scANVI** model below). Note the warning about making sure that the X field contains unnormalised count data. In our case this is expected as some of our samples contain inferred counts (as mentioned above) but usually when you see this you should check that the layer provided to the setup function does actually contain count values.

In [ ]:
model_scvi = scvi.model.SCVI(adata_scvi)
model_scvi

The model object contains the provided AnnData object as well as the neural network for the model itself. If we wanted to modify the structure of the network we could provide additional arguments to the model construction function but here we just use the defaults.

### Training the model

The model will be trained for a given number of _epochs_ (a training iteration where every cell is passed through the network). The following code is a suggested heuristic for reducing the number of epochs for larger datasets. Because this dataset has less than 20000 cells it returns the default value of 400.

In [ ]:
max_epochs_scvi = np.min([round((20000 / adata.n_obs) * 400), 400])
max_epochs_scvi

We now train the model for the selected number of epochs (this will take ~15-30 minutes depending on the computer you are using).

In [ ]:
model_scvi.train(max_epochs=max_epochs_scvi)

> **Early stopping**
>
> An alternative to manually setting the number of epochs is to set `early_stopping=True` in the training function. This will let **scVI** decide to stop training early depending on the convergence of the model.
> The exact conditions for stopping can be controlled by other parameters.

## Extracting the embedding 

The main result we want to extract from the trained model is the latent representation for each cell. This is an embedding where the batch effects have been removed. We store this in `obsm` with the key `X_scvi`.

In [ ]:
adata_scvi.obsm["X_scVI"] = model_scvi.get_latent_representation()

### Calculate corrected UMAP

We will now visualise the data as we did before integrated. We calculate a new UMAP embedding but instead of finding nearest neighbours in PCA space we start with the corrected representation from **scVI**.

In [ ]:
sc.pp.neighbors(adata_scvi, use_rep="X_scVI")
sc.tl.umap(adata_scvi)
adata_scvi

Once we have the new UMAP representation we can plot it coloured by batch and identity labels as before.

In [ ]:
sc.pl.umap(adata_scvi, color=[label_key, batch_key])

This looks much better! Before the various batches were separated and the same cell identity in different batches were far apart from each other. Now the batches are all mixed with each other and we have a single blob for each cell identity label.

In many cases we would not already have identity labels so from this stage we would continue with clustering, annotation and further analysis as described in other sections.

## Integrating with scANVI

When performing integration with **scVI** we pretended that we didn't already have any cell labels (although we showed them in plots). While this scenario is common there are some cases where we do know something about cell identity in advance. Most often this is when we want to combine one or more publicly available datasets with data from a new study. When we have labels for at least some of the cells we can used **scANVI**. This is an extension of the **scVI** model that knows about cell labels as well as batches. Because it has this extra information it can try to keep the differences between cell labels while removing batch effects. Benchmarking suggests that **scANVI** tends to better preserve biological signals compared to **scVI** but sometimes it is not as effective at removing batch effects. While we have labels for all cells here it is also possible to use **scANVI** in a semi-supervised manner where labels are only provided for some cells.

> **Label harmonization**
>
> If you are using **scANVI** to integrate multiple datasets for which you already have labels it is important to first perform _label harmonization_.
> This refers to a process of checking that labels are consistent across the datasets that are being integrated.
> How best to do this is an open question but often requires input from subject-matter experts.

We start by setting up our AnnData object as before, but now we provide a label key as well as the batch key.

In [ ]:
adata_scanvi = adata_scvi.copy()
scvi.model.SCANVI.setup_anndata(adata_scanvi, layer="counts", batch_key=batch_key, labels_key=label_key)
scvi.data.view_anndata_setup(adata_scanvi)

We can see that our setup now contains information about the cell identity labels.

The next step is now to create a **scANVI** model object. Note that now we provide the **scVI** model from the previous section rather than the AnnData object. This is because **scANVI** works by refining the model from **scVI** using the cell labels. If we had not already created and trained an **scVI** model we would need to do that first before we could use **scANVI**.

We also need to provide **scANVI** with a label for any cells without a label in the dataset. In this case all our cells are labelled so we just provide a dummy value but often it is important to make sure this is correct.

In [ ]:
model_scanvi = scvi.model.SCANVI.from_scvi_model(model_scvi, adata=adata_scanvi, unlabeled_category="unlabelled")
model_scanvi

This model object is very similar to what we saw before for **scVI**. As mentioned previously we could modify the structure of the model network but here we just use the default.

Again we have a heuristic for selecting the number of training epochs. Note that this is much fewer than before as we are just refining the **scVI** model, rather than training a whole network from scratch.

In [ ]:
max_epochs_scanvi = int(np.min([10, np.max([2, round(max_epochs_scvi / 3.0)])]))
model_scanvi.train(max_epochs=max_epochs_scanvi)

We can extract the new latent representation from the model and create a new UMAP embedding as we did for **scVI**.

In [ ]:
adata_scanvi.obsm["X_scANVI"] = model_scanvi.get_latent_representation()
sc.pp.neighbors(adata_scanvi, use_rep="X_scANVI")
sc.tl.umap(adata_scanvi)
sc.pl.umap(adata_scanvi, color=[label_key, batch_key])

By looking at the UMAP representation it is difficult to tell the difference between **scANVI** and **scVI** but when the effect of the refinement has been quantified it has been shown to often better preserve biological variation.

## BBKNN

The next method we will look at is **BBKNN** or "Batch Balanced KNN". This is a very different approach to **scVI**, rather than using a neural network to embed cells in a batch-corrected space it instead modifies how the KNN graph used for clustering and embedding is constructed. As we have seen in previous chapters the normal KNN procedure connects cells to the most similar cells across the whole dataset. The change that **BBKNN** makes is to enforce that cells are connected to cells from other batches. While this is a simple modification it can be quite effective, particularly when there are very strong batch effects. However, as the result is an integrated graph it can have limited downstream uses as few packages will accept this as an input.

An important parameter for **BBKNN** is the number of neighbours per batch. A suggested heuristic for this is to use 25 if there are less than 10000 cells or the default of 3 if there are more than 10000.

In [ ]:
neighbors_within_batch = 25 if adata_hvg.n_obs > 10000 else 3

Before using **BBKNN** we first perform a PCA as we would before building a normal KNN graph. Unlike **scVI** which models raw counts here we start with the log-normalised expression matrix.

In [ ]:
adata_bbknn = adata_hvg.copy()
adata_bbknn.X = adata_bbknn.layers["logcounts"]
sc.pp.pca(adata_bbknn)

We can now run **BBKNN**, replacing the call to the **scanpy** `neighbors()` function in a standard workflow. An important difference is to make sure the `batch_key` argument is set which specifies a column in `adata_hvg.obs` which contains batch labels.

In [ ]:
bbknn.bbknn(adata_bbknn, batch_key=batch_key, neighbors_within_batch=neighbors_within_batch)
adata_bbknn

Unlike the default **scanpy** function **BBKNN** does not allow specifying a key for storing results so they are always stored under the default "neighours" key.

We can use this new integrated graph just like we would a normal KNN graph to construct a UMAP embedding.

In [ ]:
sc.tl.umap(adata_bbknn)
sc.pl.umap(adata_bbknn, color=[label_key, batch_key])

This integration is also much improved compared to the unintegrated data with cell identities grouped together and batches clearly mixed.

## ComBat

Some downstream applications cannot accept an integrated embedding or neighbourhood graph and require a corrected expression matrix. One method that can produce this output is **ComBat**. This is an old method originally designed for microarray data that uses an empirical Bayes approach to centre and scale the expression matrix. While this is not usually among the best methods in benchmarking studies it has the advantages of being very quick to run, easier to interpret (due to a linear correction) and producing a corrected expression matrix.

There are several implementations of **ComBat** available, here we use the one in **scanpy**. Using **ComBat** is very simple, we just supply an AnnData object and a key specifying the batch variable to remove. Note that **ComBat** expects log-normalised data as input (there is a newer variant which operates directly on counts called **ComBat-seq** but we don't show that here). Once we have the corrected expression matrix we store it as a new layer called "combat".

In [ ]:
adata_combat = adata_hvg.copy()
adata_combat.X = adata_combat.layers["logcounts"]
sc.pp.combat(adata_combat, key=batch_key, inplace=True)
adata_combat.layers["combat"] = adata_combat.X
adata_combat

The corrected expression matrix can then be analysed as normal, except that now the batch effects should be removed. Let's make a UMAP as we have done for the other methods.

In [ ]:
sc.pp.pca(adata_combat)
sc.pp.neighbors(adata_combat)
sc.tl.umap(adata_combat)
sc.pl.umap(adata_combat, color=[label_key, batch_key])

While the cell labels are now better grouped the mixing between batches is not as complete as we have seen with other methods. However, while looking at a UMAP can tell you if there are very clear batch effects it is not the best way to evaluate an integration and it is important not to over interpret it. In the next section we present some approaches to more rigorously evaluate integration methods.

## Benchmarking your own integration

* How to use scIB
* Interpreting benchmarks?
* Probably just text (hard to run all the methods, could maybe compare whatever methods are run)

In [ ]:
metrics_scvi = scib.metrics.metrics_fast(adata, adata_scvi, batch_key, label_key, embed="X_scVI")
metrics_scanvi = scib.metrics.metrics_fast(adata, adata_scanvi, batch_key, label_key, embed="X_scANVI")
metrics_bbknn = scib.metrics.metrics_fast(adata, adata_bbknn, batch_key, label_key)
metrics_combat = scib.metrics.metrics_fast(adata, adata_combat, batch_key, label_key)
metrics_hvg = scib.metrics.metrics_fast(adata, adata_hvg, batch_key, label_key)

In [ ]:
# Concatenate metrics results
metrics = pd.concat([metrics_scvi, metrics_scanvi, metrics_bbknn, metrics_combat, metrics_hvg], axis="columns")
# Set methods as column names
metrics = metrics.set_axis(["scVI", "scANVI", "BBKNN", "ComBat", "Unintegrated"], axis="columns")
# Set only the fast metrics
metrics = metrics.loc[["ASW_label", "ASW_label/batch", "PCR_batch", "isolated_label_silhouette", "graph_conn", "hvg_overlap"], :]
# Transpose so that metrics are columns and methods are rows
metrics = metrics.T
metrics

In [ ]:
metrics.style.background_gradient(cmap="Blues")

In [ ]:
metrics_scaled = (metrics - metrics.min()) / (metrics.max() - metrics.min())
metrics_scaled.style.background_gradient(cmap="Blues")

In [ ]:
metrics_scaled["Batch"] = metrics_scaled[["ASW_label/batch", "PCR_batch", "graph_conn"]].mean(axis=1)
metrics_scaled["Bio"] = metrics_scaled[["ASW_label", "isolated_label_silhouette", "hvg_overlap"]].mean(axis=1)
metrics_scaled.style.background_gradient(cmap="Blues")

In [ ]:
fig, ax = plt.subplots()
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
metrics_scaled.plot.scatter(
    x="Batch",
    y="Bio",
    c=range(len(metrics_scaled)),
    colormap=matplotlib.cm.get_cmap("Set1"),
    ax=ax
)

for k, v in metrics_scaled[["Batch", "Bio"]].iterrows():
    ax.annotate(
        k, v,
        xytext=(6, -3),
        textcoords="offset points",
        family="sans-serif",
        fontsize=12
    )

In [ ]:
metrics_scaled["Overall"] = 0.4 * metrics_scaled["Batch"] + 0.6 * metrics_scaled["Bio"]
metrics_scaled.style.background_gradient(cmap="Blues")

In [ ]:
metrics_scaled.plot.bar(y = "Overall")

## Session information

In [ ]:
import session_info

session_info.show()